In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
random_state = 7
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in" 

tr = "testset_breast_healthy_20_target_in"
ts = "testset_breast_healthy_20_target_in"


# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [35]:
import pickle
with open("genes_to_keep.pickle", 'rb') as f:
    keep_pickle = pickle.load(f)

keep = []
for item in keep_pickle:
    keep.append(item)



keep.append("target")

In [36]:
len(keep)

21

In [37]:
df_reduced_train = df_whole_train[keep]
df_reduced_test = df_whole_test[keep]
df_reduced_test

,CDIPT,SGTA,SPEN,UNC119B,GNG11,MAFG,CSF2RB,ANXA11,BOLA2B,HNF1A,...,PKN2,SP3,PCM1,ZNF768,YIPF2,LRRFIP1,EDEM3,EXD3,TAF1D,target
0,9.93679,6.55449,7.79035,7.97046,12.44360,5.83029,9.82223,8.07926,9.02293,3.76997,...,6.80386,9.89520,5.57384,5.99488,4.46338,6.32727,8.94158,5.26027,8.72404,1
1,10.23390,6.72037,8.16615,8.30926,11.17150,6.85606,10.46500,8.58891,9.16681,3.96114,...,8.16149,9.77974,6.54700,5.80392,4.89965,6.94390,7.94474,5.81944,9.88740,0
2,9.67147,7.75739,10.13210,8.51146,6.72041,8.40674,11.42550,8.23108,9.40064,4.18989,...,8.30698,7.83341,7.77629,6.95394,4.87870,5.74570,7.56270,6.19068,11.01570,0
3,10.59610,5.97667,9.67245,8.09547,9.77895,6.84067,11.72110,8.75533,7.92071,4.12633,...,9.62033,7.98295,5.48364,5.91977,4.98132,8.94673,9.25207,6.04691,9.74904,0
4,9.94956,8.06992,9.83529,9.58084,6.97102,8.00435,10.77690,8.63830,9.04782,4.16948,...,8.03245,8.39622,6.74834,6.89012,5.03811,5.94280,8.14958,5.89728,10.79360,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,10.21810,6.24528,9.00733,8.34903,8.62004,6.49896,11.42390,8.47615,8.55576,4.40840,...,9.74480,8.31548,5.65292,5.36271,4.30803,8.51244,9.74927,5.38237,10.27790,0
200,9.77319,6.69310,8.94843,7.81246,10.82660,6.44281,10.19310,8.30931,8.01144,4.19677,...,8.77273,7.90687,6.03777,5.70350,4.79954,9.04800,8.85851,5.85144,9.46293,0
201,10.53870,6.96956,8.19466,8.62926,10.59430,6.56629,11.83710,8.42011,8.30606,4.24847,...,9.34497,8.89672,5.14652,5.40445,5.14894,8.00036,9.41468,5.22342,8.79654,0
202,9.74043,6.39563,7.34592,7.73972,11.42480,5.87412,9.42657,7.77638,9.18634,3.46570,...,7.14892,9.60991,4.76539,5.47906,4.64367,6.79103,9.05442,5.11422,8.56776,1


In [38]:
df_reduced_train.to_csv(parent + '\Data\\' + "trainset_breast_25.csv", index=False)
df_reduced_test.to_csv(parent + '\Data\\' + "testset_breast_25.csv", index=False)

# AML vs Healthy not in literature

In [6]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in" 

tr = "trainset_AML_vs_Healthy_100"
ts = "testset_AML_vs_Healthy_100"


# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [7]:
df_train_not_literature = df_whole_train.drop(["HLA-E", "TGFBI", "MYBL2", "CD14", "NUP214", "EPS8", "IGF1R", "PRAME", "P2RX4", "HOXB6", "WT1", "FLT3", "CDKN2A", "GPR65", "HOXA1", "LPAR6", "SMYD3", "DEPTOR", "VPREB1", "GDF15", "TCL1A", "COMMD3-BMI1"], axis=1)
df_test_not_literature = df_whole_test.drop(["HLA-E", "TGFBI", "MYBL2", "CD14", "NUP214", "EPS8", "IGF1R", "PRAME", "P2RX4", "HOXB6", "WT1", "FLT3", "CDKN2A", "GPR65", "HOXA1", "LPAR6", "SMYD3", "DEPTOR", "VPREB1", "GDF15", "TCL1A", "COMMD3-BMI1"], axis=1)

In [10]:
df_train_not_literature

,CCL5,RCN1,TRAFD1,APOC4-APOC2,TFF3,FPR1,FAM50B,GP1BB,EDIL3,GPR171,...,C3orf14,CXorf57,SLC27A5,PCDH9,NUDT11,VPREB3,PDZRN4,CLIC4P1,SH3BP4,target
0,9.772010,6.588610,7.487040,6.326160,6.359890,9.648560,6.193470,7.903320,3.075170,4.681430,...,3.960270,4.114090,6.756580,4.839050,4.800110,5.886420,2.986770,7.641210,4.600700,1
1,12.072500,7.035030,8.233320,5.780480,4.732280,9.935750,6.642010,8.414960,2.933160,9.061420,...,6.831890,6.213470,5.861720,6.898410,4.459540,7.451090,2.799260,6.413990,5.422700,0
2,9.558740,6.770510,6.758310,7.739870,6.340060,10.175600,5.816300,4.694870,3.036580,4.223230,...,4.516830,4.530420,5.674300,5.051260,5.082740,5.260470,3.121190,6.674230,3.975860,1
3,6.950090,8.024300,7.324550,8.655490,5.561650,5.447670,5.877480,6.625360,3.043830,5.310090,...,3.426270,3.508890,7.182940,4.519230,4.239360,4.130450,2.934840,6.354010,4.290930,1
4,8.067420,7.903250,6.971870,7.591880,5.916340,5.714020,5.066440,4.982270,2.987980,4.277940,...,4.191190,3.944960,6.844500,4.893830,4.277120,5.186280,3.081330,6.297370,3.566480,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036,11.535600,5.841640,8.362650,6.265170,5.463510,12.040600,6.637650,6.750340,2.990870,10.093900,...,5.724990,5.895620,5.960460,5.899420,4.303560,6.280060,3.109390,6.222340,5.155880,0
4037,7.116190,7.292161,7.608784,7.586739,6.667311,7.798482,7.043749,8.237169,4.734801,5.525074,...,4.767663,4.326941,6.574681,6.122023,7.115361,6.024381,5.394125,7.001379,6.591970,1
4038,6.662620,7.339010,6.798800,6.993970,5.790360,6.086010,6.043050,5.025080,3.203680,4.373900,...,6.204770,5.069040,6.562910,6.847660,7.071540,5.900320,3.338040,5.741800,6.296620,1
4039,5.530470,6.805010,7.003470,8.352910,5.524330,5.679400,5.509760,7.239470,3.036620,4.906100,...,3.944090,4.006060,6.359700,5.034160,5.514870,5.309800,3.030140,6.846330,4.138060,1


In [9]:
df_train_not_literature.to_csv(parent + '\Data\\' + "trainset_AML_vs_Healthy_100_not_in_literature.csv", index=False)
df_test_not_literature.to_csv(parent + '\Data\\' + "testset_AML_vs_Healthy_100_not_in_literature.csv", index=False)

# ALL vs Healthy not in literature

In [11]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in" 

tr = "trainset_ALL_vs_Healthy_150"
ts = "testset_ALL_vs_Healthy_150"


# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [12]:
df_train_not_literature = df_whole_train.drop(["STMN1" , "SOX4" ,"TGFBI" ,"AEBP1" ,"TSPAN7" ,"S100A13" ,"LYN" ,"CD93" ,"SIRPA" ,"SSBP2" ,"MAP3K5" ,"MPO" ,"P2RX4" ,"BCHE" ,"CTSG" ,"ADAM28" ,"NPY" ,"RHAG" ,"ARG1" ,"CEACAM8" ,"CCR2" ,"MMP8" ,"PSMA6" ,"THY1" ,"SLC1A5" ,"SPP1" ,"DNTT" ,"IGHV3-23" ,"ZNF423" ,"ADAM10" ,"ADA" ,"LPAR6" ,"CD244" ,"ARPP21"], axis=1)
df_test_not_literature = df_whole_test.drop(["STMN1" , "SOX4" ,"TGFBI" ,"AEBP1" ,"TSPAN7" ,"S100A13" ,"LYN" ,"CD93" ,"SIRPA" ,"SSBP2" ,"MAP3K5" ,"MPO" ,"P2RX4" ,"BCHE" ,"CTSG" ,"ADAM28" ,"NPY" ,"RHAG" ,"ARG1" ,"CEACAM8" ,"CCR2" ,"MMP8" ,"PSMA6" ,"THY1" ,"SLC1A5" ,"SPP1" ,"DNTT" ,"IGHV3-23" ,"ZNF423" ,"ADAM10" ,"ADA" ,"LPAR6" ,"CD244" ,"ARPP21"], axis=1)

In [13]:
df_train_not_literature.to_csv(parent + '\Data\\' + "trainset_ALL_vs_Healthy_150_not_in_literature.csv", index=False)
df_test_not_literature.to_csv(parent + '\Data\\' + "testset_ALL_vs_Healthy_150_not_in_literature.csv", index=False)

In [14]:
df_train_not_literature

,WARS,SDCBP,CST3,COL3A1,RER1,MATN2,IFI27,DBN1,C1QB,SMARCD1,...,MED28,TMEM106B,DPPA4,L2HGDH,VRTN,FOXRED2,ASTE1,SEMA4C,PRR5,target
0,7.592169,8.838503,6.151441,5.382015,6.986188,4.224580,6.065428,7.298434,5.819314,7.216487,...,6.744962,4.534215,4.482342,5.210566,7.992622,7.182100,6.811301,7.154131,7.747942,1
1,7.155810,8.598000,5.811540,5.570950,5.995070,3.248400,5.778960,7.749000,4.179420,7.505220,...,5.194250,6.054520,3.779860,5.581250,7.293750,7.037100,5.630670,8.184840,6.182200,1
2,8.024406,9.603805,7.271123,5.090965,7.605913,4.121849,6.723201,8.322849,5.595167,7.531628,...,8.045335,5.530472,4.431018,5.140376,6.653190,7.246701,6.717686,6.786491,7.593878,1
3,8.384836,8.025827,5.443172,5.663897,7.235209,4.660694,8.245308,7.919013,6.424624,6.638817,...,8.128368,4.525453,4.700825,4.878005,7.846647,5.792978,7.379145,6.545153,7.576077,1
4,10.425700,11.747700,11.395600,4.267280,8.705490,3.450720,4.897780,6.669080,4.627040,7.340240,...,6.068870,8.312920,3.308290,4.947730,5.746580,6.158910,6.388380,7.674050,7.392490,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,8.482300,9.995340,5.854950,4.348700,9.123270,3.120460,4.635130,8.183680,4.243090,8.120660,...,7.790490,7.861220,3.081050,5.650910,5.979150,6.903630,6.572240,6.233320,6.300700,1
3494,8.622109,9.834865,8.677433,5.207763,7.678730,4.261137,5.861557,7.474985,5.491992,6.807649,...,7.969268,5.365993,6.118198,5.042093,6.288468,6.665375,7.448243,7.820779,8.212433,0
3495,8.196201,9.926119,7.577701,6.877518,8.485634,3.961185,8.732912,8.759654,7.129287,6.564231,...,8.889535,6.361668,4.364104,4.706063,6.767586,5.965524,7.142933,6.580128,6.728891,1
3496,8.392720,8.161030,7.076030,4.089110,7.220750,3.277080,5.446460,9.366650,4.337540,8.123340,...,5.256730,6.853560,3.572970,5.408070,6.313730,6.857840,6.575740,6.089020,6.452180,1


In [17]:
df_train_not_literature

,WARS,SDCBP,CST3,COL3A1,RER1,MATN2,IFI27,DBN1,C1QB,SMARCD1,...,MED28,TMEM106B,DPPA4,L2HGDH,VRTN,FOXRED2,ASTE1,SEMA4C,PRR5,target
0,7.592169,8.838503,6.151441,5.382015,6.986188,4.224580,6.065428,7.298434,5.819314,7.216487,...,6.744962,4.534215,4.482342,5.210566,7.992622,7.182100,6.811301,7.154131,7.747942,1
1,7.155810,8.598000,5.811540,5.570950,5.995070,3.248400,5.778960,7.749000,4.179420,7.505220,...,5.194250,6.054520,3.779860,5.581250,7.293750,7.037100,5.630670,8.184840,6.182200,1
2,8.024406,9.603805,7.271123,5.090965,7.605913,4.121849,6.723201,8.322849,5.595167,7.531628,...,8.045335,5.530472,4.431018,5.140376,6.653190,7.246701,6.717686,6.786491,7.593878,1
3,8.384836,8.025827,5.443172,5.663897,7.235209,4.660694,8.245308,7.919013,6.424624,6.638817,...,8.128368,4.525453,4.700825,4.878005,7.846647,5.792978,7.379145,6.545153,7.576077,1
4,10.425700,11.747700,11.395600,4.267280,8.705490,3.450720,4.897780,6.669080,4.627040,7.340240,...,6.068870,8.312920,3.308290,4.947730,5.746580,6.158910,6.388380,7.674050,7.392490,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,8.482300,9.995340,5.854950,4.348700,9.123270,3.120460,4.635130,8.183680,4.243090,8.120660,...,7.790490,7.861220,3.081050,5.650910,5.979150,6.903630,6.572240,6.233320,6.300700,1
3494,8.622109,9.834865,8.677433,5.207763,7.678730,4.261137,5.861557,7.474985,5.491992,6.807649,...,7.969268,5.365993,6.118198,5.042093,6.288468,6.665375,7.448243,7.820779,8.212433,0
3495,8.196201,9.926119,7.577701,6.877518,8.485634,3.961185,8.732912,8.759654,7.129287,6.564231,...,8.889535,6.361668,4.364104,4.706063,6.767586,5.965524,7.142933,6.580128,6.728891,1
3496,8.392720,8.161030,7.076030,4.089110,7.220750,3.277080,5.446460,9.366650,4.337540,8.123340,...,5.256730,6.853560,3.572970,5.408070,6.313730,6.857840,6.575740,6.089020,6.452180,1
